In [21]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

base = np.array([[20],[30],[50],[120],[22],[63],[102],[34],[87],[12],[25],[128],[23],[77],[56]])
target = np.array([[50],[70],[110],[250],[54],[136],[214],[78],[184],[34],[60],[266],[56],[166],[122]])
scaler = MinMaxScaler()
base_scaled = scaler.fit_transform(base)
base_unscaled = scaler.inverse_transform(base_scaled)

target_scaled = scaler.fit_transform(target[:5])
target_unscaled = scaler.inverse_transform(target_scaled)

num_clusters = 3
#print(target_scaled)

/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [22]:

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3)
kmeans.fit(base_scaled)

base_centroids = kmeans.cluster_centers_
#labels = kmeans.labels_
    
print(kmeans.cluster_centers_)
print(kmeans.inertia_)

[[ 0.10098522]
 [ 0.90229885]
 [ 0.47068966]]
0.117515995697


In [23]:
def cal_centroids(alpha,beta):

    target_centroids = []
    for i in range(len(base_centroids)):
        val = alpha*base_centroids[i][0] + beta
        target_centroids.append([val])

    target_centroids = np.array(target_centroids)
    print("new centroids: ", target_centroids)
    return target_centroids
    #print(target_centroids)

In [40]:
import numpy as np
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [42]:
# def closest_centroids(old_centroids,new_centroids):
#     final_centroids = np.zeros(num_clusters)
#     for old_center in old_centroids:
#         index = find_nearest(old_center,new_centroids)
#         final_centroids[index] = old_center
        
#     print(final_centroids)    
#     return final_centroids
        

In [61]:
def closest_centroids(old_centroids,new_centroids):
    final_centroids = []
    for i in range(num_clusters):
        final_centroids.append([])
        
    
    for i in range(num_clusters):
        min_idx = -1
        min_value = 2
        for j in range(num_clusters):
            diff = abs(old_centroids[i][0] - new_centroids[j][0])
            if(diff < min_value):
                idx = j
                min_value = diff
        
        print(old_centroids[i][0],"closest: ", new_centroids[idx][0])
        final_centroids[idx] = old_centroids[i]
        
            
        
    print(final_centroids)    
    return final_centroids

In [62]:
def find_members(centroids,labels):
    #centroids = kmeans.cluster_centers_
    #labels = kmeans.labels_

    #c = Counter(labels)

    clusters = []
    for i in range(0,num_clusters):
        clusters.append([])

    #print(kmeans.cluster_centers_)
    for i in range(len(target_scaled)):
        #print("coordinate:",X[i], "label:", labels[i], "keyword:", keywords[i])
        clusters[labels[i]].append(target_scaled[i])
    print("members: ",clusters)    
    return clusters

In [72]:
def cal_gradient(clusters,reframed_centroids,old_alpha,old_beta):
    gradient_alpha = 0
    gradient_beta = 0
    for i in range(0,num_clusters):
        gradient = 0
        print(reframed_centroids[i])
        print(centroids[i])
        for member in clusters[i]:
            print("ghghh")
            print(member[0])
            gradient =  (reframed_centroids[i][0] - member[0])
            gradient_alpha = gradient_alpha + gradient*reframed_centroids[i][0]
            gradient_beta = gradient_beta + gradient

#         gradient_alpha = gradient_alpha + gradient*reframed_centroids[i][0]
#         gradient_beta = gradient_beta + gradient
#         print(clusters[i])
#         print(gradient_alpha)
#         print(gradient_beta)
        print("******")
    print(gradient_alpha,gradient_beta)    
    new_alpha = old_alpha-1*gradient_alpha
    new_beta = old_beta-1*gradient_beta
    return [new_alpha,new_beta]

In [75]:
sum_m = 0
sum_d = 0
for i in range(len(base_centroids)):
    sum_m = sum_m + base_centroids[i][0]
    
avg_m = sum_m/len(base_centroids)

for i in range(len(target_scaled)):
    sum_d = sum_d + target_scaled[i][0]

print(sum_m,sum_d)
avg_d = sum_d/len(target_scaled)

alpha = avg_d/avg_m
beta = 0
print(alpha)
        


1.47397372742 1.42
0.578029298724


In [76]:
reframed_centroids =  cal_centroids(alpha,beta)
kmeans = KMeans(n_clusters=num_clusters, init=reframed_centroids, max_iter=1)
kmeans.fit(target_scaled)
old_error = kmeans.inertia_
centroids = kmeans.cluster_centers_
labels = kmeans.labels_


while(1):
    print(centroids)
    print("old error: ",old_error)
    
    clusters = find_members(centroids,labels)
    
    reframed_centroids = closest_centroids(reframed_centroids,centroids)
    
    new_alphabeta = cal_gradient(clusters,reframed_centroids,alpha,beta)
    
    alpha = new_alphabeta[0]
    beta = new_alphabeta[1]
    
    print("new alpha beta", alpha, beta)
    
    reframed_centroids =  cal_centroids(alpha,beta)
    
    kmeans = KMeans(n_clusters=num_clusters, init=reframed_centroids, max_iter=1)
    kmeans.fit(target_scaled)
    new_error = kmeans.inertia_
    
    print("compare ",old_error,new_error)
    if(new_error < old_error):
        old_error = new_error
                
    else:
        break; 

    #base_centroids = kmeans.cluster_centers_
    #labels = kmeans.labels_

    #print(kmeans.cluster_centers_)
    #print(old_error,new_error)
    
print("finalparameters", alpha,beta)

new centroids:  [[ 0.05837242]
 [ 0.52155517]
 [ 0.27207241]]
[[ 0.04]
 [ 1.  ]
 [ 0.3 ]]
old error:  0.0056
members:  [[array([ 0.]), array([ 0.1]), array([ 0.02])], [array([ 1.])], [array([ 0.3])]]
0.0583724168663 closest:  0.04
0.521555171838 closest:  0.3
0.272072411296 closest:  0.3
[array([ 0.05837242]), [], array([ 0.27207241])]
[ 0.05837242]
[ 0.04]
ghghh
0.0
ghghh
0.1
ghghh
0.02
******
[]
[ 1.]
ghghh
1.0


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


IndexError: list index out of range

In [7]:
from collections import Counter

centroids = kmeans.cluster_centers_
labels = kmeans.labels_

c = Counter(labels)

clusters = []
for i in range(0,3):
    clusters.append([])
    
#print(kmeans.cluster_centers_)
for i in range(len(target_scaled)):
    #print("coordinate:",X[i], "label:", labels[i], "keyword:", keywords[i])
    clusters[labels[i]].append(target_scaled[i])

#unscaled_centroids = scaler.inverse_transform(centroids)
gradient_alpha = 0
gradient_beta = 0
for i in range(0,3):
    gradient = 0
    print(reframed_centroids[i][0])
    print(centroids[i])
    for member in clusters[i]:
        print(member[0])
        gradient = reframed_centroids[i][0] - member[0]
    
    gradient_alpha = gradient_alpha + gradient*reframed_centroids[i][0]
    gradient_beta = gradient_beta + gradient
    print(clusters[i])
    print(gradient_alpha)
    print(gradient_beta)
    print("******")

0.8143006871988968
[1.]
1.0
[array([1.])]
-0.15121507802630121
-0.1856993128011032
******
0.17470606057119187
[0.105]
0.0
0.10000000000000003
0.30000000000000004
0.020000000000000018
[array([0.]), array([0.1]), array([0.3]), array([0.02])]
-0.12418699163742009
-0.03099325222991134
******
1.5609915493312188
[1.]
[]
-0.12418699163742009
-0.03099325222991134
******


In [8]:
alpha = alpha-1*gradient_alpha
beta = beta-1*gradient_beta
print(alpha,beta)

1.8542031036350763 0.03099325222991134


In [9]:
reframed_centroids =  cal_centroids(alpha,beta)
kmeans = KMeans(n_clusters=num_clusters, init=reframed_centroids, max_iter=1)
kmeans.fit(target_scaled)
old_error = kmeans.inertia_
print(old_error)

[[0.90374747]
 [0.21824036]
 [1.70403858]]
0.03845


c:\users\user\appdata\local\programs\python\python36-32\lib\site-packages\sklearn\cluster\k_means_.py:896: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
